# Network/Flow Accumulation Covariates
Covariates to be calculated along across the flow accumulation grids:
1. fac_glacier_per - % Glacier along flow accumulation = glacier weighted fac/fac
2. fac_lake_per	- % Lake along flow accumulation = lake weighted fac/fac
3. fac_mn_slope - Mean slope contributing area = slope weighted fac/fac
4. fac_north_per - % North along flow accumulation	north weighted fac/fac
5. fac_wetland_per - % Wetland along flow accumulation = wetland weighted fac/fac
6. fac_mn_lcld_xxxx - Mean day of year for LCLD (by year) along flow accumulation = mdy(year) weighted fac/fac

In [5]:
###
# Import modules
import datetime
import arcpy
import sys
import os

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'sys paths {sys.path}')
today = datetime.date.today()
print ('ctime:', today.ctime())


imports complete
sys paths ['C:\\Program Files\\JetBrains\\PyCharm 2020.2.3\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Program Files\\JetBrains\\PyCharm 2020.2.3\\plugins\\python\\helpers\\pydev', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmer

## Collect datasets
Store the following in a data dictionary using region as the key
1. Flow accumulation grids
2. Slope - Slope in degrees - float
3. Wetlands - Binary raster data
4. Lakes/Ponds - Feature class convert to raster and
5. Glaciers - Binary raster data
6. North - Binary raster data
7. LCLD rasters - integertype


In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [21]:
arcpy.env.overwriteOutput = True

# Set data dir equal to directory containing the AKSSF regional sub-folders.
data_dir = r"D:\\GIS\\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
# Create dictionary to store scratch workspaces
scrDict = {}
# Regions to be processed
rList = ['Kodiak','Prince_William_Sound','Copper_River','Bristol_Bay','Cook_Inlet']
regions = arcpy.ListWorkspaces(workspace_type="Folder")
regions = [os.path.normpath(r) for r in regions if os.path.basename(r) in rList]

#working gdb to save intermediate outputs
ca_scratchgdb = r"U:\\ca_Outputs\\ca_scratch.gdb"
ca_scratchfol = r"U:\\ca_Outputs"
print (regions)

#Create rvbd scratch folders if they do not already exist
import os, arcpy


if not arcpy.Exists(ca_scratchfol):
    os.mkdir(ca_scratchfol)
    print(f'Creating scratch folder {ca_scratchfol}')
else:
    print(f'Scratch folder {ca_scratchfol} already exists')
print('----------')

if not arcpy.Exists(ca_scratchgdb):
    arcpy.CreateFileGDB_management(ca_scratchfol,'ca_scratch.gdb')
    print(f'Creating scratch gdb {ca_scratchgdb}')
else:
    print(f'Scratch folder {ca_scratchgdb} already exists')
print('----------')

#Create Regional output folders
for region in regions:
    rname = os.path.basename(region)
    cascratchpath = os.path.join(ca_scratchfol, rname + '_ca')
    append_value(scrDict,rname,cascratchpath)
    if not arcpy.Exists(cascratchpath):
        os.mkdir(cascratchpath)
        print (f'{rname} does not have a scratch folder')
        print (f'Creating contributing area scratch folder at  {cascratchpath}')
    else:
        print(f'Scratch folder for {rname} already created at {cascratchpath}')

    print('----------')
print(f'All scratch workspaces set')
scrDict

['D:\\GIS\\AKSSF\\Bristol_Bay', 'D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River', 'D:\\GIS\\AKSSF\\Kodiak', 'D:\\GIS\\AKSSF\\Prince_William_Sound']
Scratch folder U:\\ca_Outputs already exists
----------
Scratch folder U:\\ca_Outputs\\ca_scratch.gdb already exists
----------
Scratch folder for Bristol_Bay already created at U:\\ca_Outputs\Bristol_Bay_ca
----------
Scratch folder for Cook_Inlet already created at U:\\ca_Outputs\Cook_Inlet_ca
----------
Scratch folder for Copper_River already created at U:\\ca_Outputs\Copper_River_ca
----------
Scratch folder for Kodiak already created at U:\\ca_Outputs\Kodiak_ca
----------
Scratch folder for Prince_William_Sound already created at U:\\ca_Outputs\Prince_William_Sound_ca
----------
All scratch workspaces set


{'Bristol_Bay': 'U:\\\\ca_Outputs\\Bristol_Bay_ca',
 'Cook_Inlet': 'U:\\\\ca_Outputs\\Cook_Inlet_ca',
 'Copper_River': 'U:\\\\ca_Outputs\\Copper_River_ca',
 'Kodiak': 'U:\\\\ca_Outputs\\Kodiak_ca',
 'Prince_William_Sound': 'U:\\\\ca_Outputs\\Prince_William_Sound_ca'}

### Iterate through regions and pull out glacier polygons and nhd waterbodies and convert to raster.  Use elev raster as snap raster/mask for these processes


### Set data dictionary and create/modify inputs as necessary

In [1]:
import arcpy, os, datetime, time
from arcpy.sa import *
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
# set pp factor
arcpy.env.parallelProcessingFactor = "80%"

# Path to lakes data for regions without NHDPlus data
sourcelakes = r"D:\\Basedata\\NHD_H_Alaska_State_GDB.gdb\\Hydrography\\NHDWaterbody"

# List to store fac grids for data that need to be iterated on grid by grid basis (mdy LCLD)
fdr_list = []
fac_list = []

caDict = {}

# Walk through gdbs and set input data sets. Create necessary raster datasets if they do not already exist.
akssf_gdb = r'D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb'
start = datetime.datetime.now()
for region in regions:
    # Start iter timing function
    iteration_start = time.time()
    rname = os.path.basename(region)
    print(rname)
    arcpy.env.workspace = region
    arcpy.env.snapRaster = os.path.join(region, 'fac.tif')
    arcpy.env.cellSize = os.path.join(region, 'fac.tif')
    arcpy.env.mask = os.path.join(region, 'fac.tif')
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    outgdb = gdb[0]
    walk = arcpy.da.Walk(region, topdown = True, datatype=['RasterDataset','FeatureClass'])
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            # Set glacier polygon fc
            if 'glaciers' == filename:
                glacfc_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,glacfc_path)

            # Extract region buffer
            elif 'region_buf' == filename:
                rbuff_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,rbuff_path)

            # Set watersheds - Dissolve watershed merge if not already created
            elif 'wtds_merge' == filename:
                wtdsource = os.path.join(dirpath, filename)
                # Dissolve Watersheds - keep as single part
                wtd_dis = os.path.join(outgdb,'wtds_dis')
                append_value(caDict, rname, wtd_dis)
                if not arcpy.Exists(os.path.join(outgdb,'wtds_dis')):
                    print('Dissolving watersheds')
                    arcpy.Dissolve_management(wtdsource,wtd_dis,None, None, "SINGLE_PART", "DISSOLVE_LINES")

            # Extract cats intersect
            elif 'cats_intersect' == filename:
                cats_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,cats_path)

            # Set stream polygon fc
            elif 'streams_merge' == filename or 'NHDFlowline_merge' == filename:
                strsource = os.path.join(dirpath, filename)
                append_value(caDict,rname,strsource)

            # Set merged watersheds dataset
            elif 'wtds_merge' == filename:
                wtdsource = os.path.join(dirpath, filename)
                # Append watersheds to dictionary
                append_value(caDict,rname,wtdsource)

            # Set elev raster
            elif 'elev.tif' == filename:
                elev_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,elev_path)

            # Set ca raster
            elif 'fac.tif' == filename:
                fac_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,fac_path)
                fac_list.append(fac_path)

            # Set flow direction raster
            elif 'fdr.tif' == filename:
                fdr_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,fdr_path)
                fdr_list.append(fdr_path)

            # Set slope raster
            elif 'slope.tif' == filename:
                slope_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,slope_path)

            # Set North raster
            elif 'north.tif' == filename:
                nor_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,nor_path)

            # Set wetland raster
            elif 'wetlands.tif' == filename:
                wras_path = os.path.join(dirpath, filename)
                append_value(caDict,rname,wras_path)

    # Check that waterbody feature class has been copied over to gdb, Originally never created for synthetic regions
    lakefc_path = os.path.join(outgdb,'NHDWaterbody_merge')
    if not arcpy.Exists(lakefc_path):
        lakesubset = arcpy.SelectLayerByLocation_management(sourcelakes,'INTERSECT',rbuff_path,'','SUBSET_SELECTION')
        print(f'{arcpy.GetCount_management(lakesubset)} Lake/ponds in {rname}...')
        lakefc_path = arcpy.FeatureClassToFeatureClass_conversion(lakesubset, outgdb, 'NHDWaterbody_merge')
    else:
        print(f'Lake dataset {lakefc_path} set for {rname}')

    # Check for lakes raster
    lake_path = os.path.join(region, 'lakes.tif')
    lakerast = os.path.join(region,'lakesras.tif')
    if not arcpy.Exists(lake_path):
        print('Lake raster not yet created')
        lakeselect = arcpy.SelectLayerByAttribute_management(lakefc_path, 'NEW_SELECTION','FTYPE = 390')
        print(f'{arcpy.GetCount_management(lakeselect)} Lake/ponds in source dataset...')
        arcpy.PolygonToRaster_conversion(lakeselect,'FType',lakerast)
        arcpy.ddd.Reclassify(lakerast, "Value", "390 1;NODATA 0", lake_path, "DATA")
        append_value(caDict,rname,lake_path)
    else:
        print(f'Lakes raster {lake_path} already exists...')
        append_value(caDict,rname,lake_path)

    # Check for glacier raster
    glac_path = os.path.join(region, 'glaciers.tif')
    glacrast = os.path.join(region,'glacrast.tif')
    if not arcpy.Exists(glac_path):
        print('Glacier raster not yet created')
        arcpy.PolygonToRaster_conversion(glacfc_path,'O1Region',glacrast)
        arcpy.ddd.Reclassify(glacrast, "Value", "1 1;NODATA 0", glac_path, "DATA")
        append_value(caDict,rname,glac_path)
    else:
        print(f'Glacier raster {glac_path} already...')
        append_value(caDict,rname,glac_path)

    print(f'Flow accumulation raster input = {fac_path} and exists = {arcpy.Exists(fac_path)}')
    print(f'Flow direction raster input = {fdr_path} and exists = {arcpy.Exists(fdr_path)}')
    print(f'Slope input = {slope_path} and exists = {arcpy.Exists(slope_path)}')
    print(f'North raster input = {nor_path} and exists = {arcpy.Exists(nor_path)}')
    print(f'Wetlands raster input = {wras_path} and exists = {arcpy.Exists(wras_path)}')
    print(f'Lakes raster input = {lake_path} and exists = {arcpy.Exists(lake_path)}')
    print(f'Glaciers raster input = {glac_path} and exists = {arcpy.Exists(glac_path)}')
    print(f'Catchments input = {cats_path} and exists = {arcpy.Exists(cats_path)}')
    print(f'Watersheds to process input = {wtd_dis} and exists = {arcpy.Exists(wtd_dis)}')


    # Stop iteration timer
    iteration_stop = time.time()
    iter_time = int (iteration_stop - iteration_start)
    print(f'Process time for {rname} complete - Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
    print('----------')
# End timer
stop = datetime.datetime.now()
print(f'Process complete {stop-start}')
print('----------')
print('Data Dictionary created')

NameError: name 'regions' is not defined

## Begin Creating weighted fac grids

Can these be run in parallel?

Example from <a href = 'https://www.e-education.psu.edu/geog489/node/2282'>PSU sequential to multiprocessing</a> and <a href='https://www.esri.com/arcgis-blog/products/analytics/analytics/multiprocessing-with-arcgis-raster-analysis/#processing-a-workflow'>Esri talk</a>

### Setup _very_ simple timing.
import time
start_time = time.time()

import arcpy
from arcpy.sa import *

import os
import multiprocessing

arcpy.env.overwriteOutput = True
arcpy.env.workspace = r'C:\489\PSU_LiDAR'

## If our rasters aren't in our filter list then drop them from our list.
def filter_list(fileList,filterList):
    return[i for i in fileList if any(j in i for j in filterList)]

def worker(raster):
    ## Note also for performance we're not saving any of the intermediate rasters - they will exist only in memory
    ## Fill the DEM to remove any sinks
    # This is the initial workspace but to prevent any file locking we'll create a scratch workspace
    # just for this raster
    arcpy.env.scratchWorkspace = os.path.join(arcpy.env.workspace,'f'+raster.replace(".img",""))  # r'C:\Users\yourname\PSU_LiDAR\f'+raster.replace(".img","")
    ## and we'll check if that scratch folder exists and if not we'll create it.
    if not os.path.exists(arcpy.env.scratchWorkspace):
        os.makedirs(arcpy.env.scratchWorkspace)
    try:
        FilledRaster = Fill(raster)
        ## Calculate the Flow Direction (how water runs across the surface)
        FlowDirRaster = FlowDirection(FilledRaster)
        ## Calculate the Flow Accumulation (where the water accumulates in the surface)
        FlowAccRaster = FlowAccumulation(FlowDirRaster)
        ## Convert the Flow Accumulation to a Stream Network
        ## We're setting an arbitrary threshold of 100 cells flowing into another cell to set it as part of our stream
        ## http://pro.arcgis.com/en/pro-app/tool-reference/spatial-analyst/identifying-stream-networks.htm
        Streams = Con(FlowAccRaster,1,"","Value > 100")
        ## Convert the Raster Stream network to a feature class
        output_Polyline = raster.replace(".img",".shp")
        arcpy.CheckOutExtension("Spatial")
        arcpy.sa.StreamToFeature(Streams,FlowDirRaster,output_Polyline)
        arcpy.CheckInExtension("Spatial")
    except:
        print ("Errors occured")
        print (arcpy.GetMessages())
        arcpy.AddMessage ("Errors occurred")
        arcpy.AddMessage(arcpy.GetMessages())

def mp_handler():

    # Ordinarily we would want all of the rasters I'm filtering by a small set for testing & efficiency
    # I did this by manually looking up the tile index for the LiDAR and determining an area of interest
    # tiles ending in 227, 228, 230, 231, 232, 233, 235, 236
    wildCardList = set(['227','228','230','231','232','233','235','236'])

    # Get a list of rasters in my folder
    rasters = arcpy.ListRasters("*")
    new_rasters = filter_list(rasters,wildCardList)

    # setup the multiprocessing pool with the number of cores in the current PC
    with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
        # use the map function of pool to call the function worker() and pass it a raster
        pool.map(worker,new_rasters)

if __name__ == '__main__':
    mp_handler()
    # Output how long the process took.
    arcpy.AddMessage("--- %s seconds ---" % (time.time() - start_time))

    print ("--- %s seconds ---" % (time.time() - start_time))
### Extract input raster by mask and store in memory to reduce processing time - setting a mask will still process entire extent, just output those cells as No Data


In [38]:
c=0
for v in caDict['Bristol_Bay']:
    print(f'{c}. {v}')
    c+=1


0. D:\GIS\AKSSF\Bristol_Bay\elev.tif
1. D:\GIS\AKSSF\Bristol_Bay\fac.tif
2. D:\GIS\AKSSF\Bristol_Bay\fdr.tif
3. D:\GIS\AKSSF\Bristol_Bay\north.tif
4. D:\GIS\AKSSF\Bristol_Bay\slope.tif
5. D:\GIS\AKSSF\Bristol_Bay\wetlands.tif
6. D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\streams_merge
7. D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\region_buf
8. D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\glaciers
9. D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\wtds_merge
10. D:\GIS\AKSSF\Bristol_Bay\lakes.tif
11. D:\GIS\AKSSF\Bristol_Bay\glaciers.tif


In [ ]:
### Begin fac calculations
import arcpy, os, datetime, time
from time import strftime
from arcpy.sa import *
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
# set pp factor
arcpy.env.parallelProcessingFactor = "100%"

# path to parent folder containing all lcld rasters - May run seperately
lcldfol = r"D:\\Basedata\\LCLD_rasters"

# Dictionary to fac covariate rasters to convert to table/merge
facDict = {}

# Walk through gdbs and set input data sets. Create necessary raster datasets if they do not already exist.
akssf_gdb = r'D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb'
process_start = time.time()

# Test on pws first
regions = ["D:\\GIS\\AKSSF\\Cook_Inlet"]

# Set huc of interest to use as mask for testing
test_huc = r"U:\\ca_Outputs\\ca_scratch.gdb\\ci_test_huc12s"

for region in regions:
    # Start iter timing function
    iteration_start = time.time()

    rname = os.path.basename(region)
    print(rname)

    # Set inputs from dictionaries
    fac_in = caDict[rname][1]
    fdr_in = caDict[rname][2]
    north_in =caDict[rname][3]
    slope_in = caDict[rname][4]
    wetlands_in = caDict[rname][5]
    glac_in = caDict[rname][11]
    lake_in = caDict[rname][10]
    scratch = scrDict[rname]

    arcpy.env.workspace = region
    arcpy.env.snapRaster = fdr_in
    arcpy.env.cellSize = 10


    # fac_glacier_per - % Glacier along flow accumulation = glacier weighted fac/fac
    start = time.time()
    print (f'Begin Process: Glacial weighting ({strftime("%m-%d-%Y %H:%M:%S")})')
    print ("")

    glacw8name = "glacw8fac.tif"
    glacpername = 'perglacfac.tif'
    glacPer_path = os.path.join(scratch, glacpername)
    append_value(facDict,rname,glacPer_path)
    if not arcpy.Exists(glacPer_path):
        glacw8flow = FlowAccumulation(fdr_in, glac_in, 'FLOAT', 'D8') # Create glacier weighted flow accumulation raster
        glacw8flow.save(os.path.join(scratch,glacw8name)) # not sure if we need to save intermediates?
        print(f'Finish glacier weighted flow accumulation at {strftime("%m-%d-%Y %H:%M:%S")}')
        glacPer = (glacw8flow/fac_in)
        glacPer.save(os.path.join(scratch,glacpername))
        print(f'Finish percent glacier raster {strftime("%m-%d-%Y %H:%M:%S")}')
    else:
        print(f'Percent glacier along flow accumulation already created {glacPer_path}')
    stop = time.time()
    elapsed = int(stop - start)
    print(f'Percent Glacier cover along FAC for {rname} complete - Elapsed time: ({datetime.timedelta(seconds=elapsed)})')
    print ("")

    # fac_lake_per - % Lake along flow accumulation = lake weighted fac/fac
    start = time.time()
    print (f'Begin Process: Lake Weighting ({strftime("%m-%d-%Y %H:%M:%S")})')
    print ("")

    lakew8name = "lakew8fac.tif"
    lakepername = 'perlakefac.tif'
    lakePer_path = os.path.join(scratch, lakepername)
    append_value(facDict,rname,lakePer_path)
    if not arcpy.Exists(lakePer_path):
        lakew8flow = FlowAccumulation(fdr_in, lake_in, 'FLOAT', 'D8') # Create lake weighted flow accumulation raster
        lakew8flow.save(os.path.join(scratch,lakew8name)) #not sure if we need to save intermediates?
        print(f'Finish lake weighted flow accumulation at {strftime("%m-%d-%Y %H:%M:%S")}')
        lakePer = (lakew8flow/fac_in)
        lakePer.save(os.path.join(scratch,lakepername))
        print(f'Finish percent lake raster {strftime("%m-%d-%Y %H:%M:%S")}')
    else:
        print(f'Percent Lake cover along flow accumulation already created {lakePer_path}')
    stop = time.time()
    elapsed = int(stop - start)
    print(f'Percent lake cover along FAC for {rname} complete - Elapsed time: ({datetime.timedelta(seconds=elapsed)})')
    print ("")

    # fac_mn_slope - Mean slope contributing area = slope weighted fac/fac
    start = time.time()
    print (f'Begin Process: Slope Weighting ({strftime("%m-%d-%Y %H:%M:%S")})')
    print ("")

    slopew8name = "slopew8fac.tif"
    slopepername = 'mnslopefac.tif'
    slopePer_path = os.path.join(scratch, slopepername)
    append_value(facDict,rname,slopePer_path)
    if not arcpy.Exists(slopePer_path):
        slopew8flow = FlowAccumulation(fdr_in, slope_in, 'FLOAT', 'D8') # Create slope weighted flow accumulation raster
        slopew8flow.save(os.path.join(scratch,slopew8name)) #not sure if we need to save intermediates?
        print(f'Finish slope weighted flow accumulation at {strftime("%m-%d-%Y %H:%M:%S")}')
        slopePer = (slopew8flow/fac_in)
        slopePer.save(os.path.join(scratch,slopepername))
        print(f'Finish mean slope raster {strftime("%m-%d-%Y %H:%M:%S")}')
    else:
        print(f'Mean slope along flow accumulation already created {slopePer_path}')
    stop = time.time()
    elapsed = int(stop - start)
    print(f'Mean slope cover along FAC for {rname} complete - Elapsed time: ({datetime.timedelta(seconds=elapsed)})')
    print ("")

    # fac_north_per - % North along flow accumulation north weighted fac/fac
    start = time.time()
    print (f'Begin Process: North Weighting ({strftime("%m-%d-%Y %H:%M:%S")})')
    print ("")

    northw8name = "northw8fac.tif"
    northpername = 'pernorthfac.tif'
    northPer_path = os.path.join(scratch, northpername)
    append_value(facDict,rname,northPer_path)
    if not arcpy.Exists(northPer_path):
        northw8flow = FlowAccumulation(fdr_in, north_in, 'FLOAT', 'D8') # Create north weighted flow accumulation raster
        northw8flow.save(os.path.join(scratch,northw8name)) #not sure if we need to save intermediates?
        print(f'Finish north weighted flow accumulation at {strftime("%m-%d-%Y %H:%M:%S")}')
        northPer = (northw8flow/fac_in)
        northPer.save(os.path.join(scratch,northpername))
        print(f'Finish percent north raster {strftime("%m-%d-%Y %H:%M:%S")}')
    else:
        print(f'Percent north cover along flow accumulation already created {northPer_path}')
    stop = time.time()
    elapsed = int(stop - start)
    print(f'Percent north cover along FAC for {rname} complete - Elapsed time: ({datetime.timedelta(seconds=elapsed)})')
    print ("")

    # fac_wetland_per - % Wetland along flow accumulation = wetland weighted fac/fac
    start = time.time()
    print (f'Begin Process: Wetland Weighting ({strftime("%m-%d-%Y %H:%M:%S")})')
    print ("")

    wetlandsw8name = "wetlandsw8fac.tif"
    wetlandspername = 'perwetlandsfac.tif'
    wetlandsPer_path = os.path.join(scratch, wetlandspername)
    append_value(facDict,rname,wetlandsPer_path)
    if not arcpy.Exists(wetlandsPer_path):
        wetlandsw8flow = FlowAccumulation(fdr_in, wetlands_in, 'FLOAT', 'D8') # Create wetlands weighted flow accumulation raster
        wetlandsw8flow.save(os.path.join(scratch,wetlandsw8name)) #not sure if we need to save intermediates?
        print(f'Finish wetlands weighted flow accumulation at {strftime("%m-%d-%Y %H:%M:%S")}')
        wetlandsPer = (wetlandsw8flow/fac_in)
        wetlandsPer.save(os.path.join(scratch,wetlandspername))
        print(f'Finish percent wetlands raster {strftime("%m-%d-%Y %H:%M:%S")}')
    else:
        print(f'Percent wetlands cover along flow accumulation already created {wetlandsPer_path}')
    stop = time.time()
    elapsed = int(stop - start)
    print(f'Percent wetlands cover along FAC for {rname} complete - Elapsed time: ({datetime.timedelta(seconds=elapsed)})')
    print ("")

    # fac_mn_lcld_xxxx - Mean day of year for LCLD (by year) along flow accumulation = mdy(year) weighted fac/fac

    walk = arcpy.da.Walk(lcldfol, datatype = ['RasterDataset'])
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            start = time.time()
            print (f'Begin Process: {filename} Weighting ({strftime("%m-%d-%Y %H:%M:%S")})')
            print ("")
            print(filename[:-4])
            rasname = filename[:-4]
            lcldw8name =  rasname +"_w8fac.tif"
            lcldpername = rasname + '_mnlcldfac.tif'
            lcldPer_path = os.path.join(scratch, lcldpername)
            lcld_in = os.path.join(dirpath,filename)
            append_value(facDict,rname,lcldPer_path)
            if not arcpy.Exists(lcldPer_path):
                lcldw8flow = FlowAccumulation(fdr_in, lcld_in, 'FLOAT', 'D8') # Create lcld weighted flow accumulation raster
                lcldw8flow.save(os.path.join(scratch,lcldw8name)) # not sure if we need to save intermediates?
                print(f'Finish lcld weighted flow accumulation at {strftime("%m-%d-%Y %H:%M:%S")}')
                lcldPer = (lcldw8flow/fac_in)
                lcldPer.save(os.path.join(scratch,lcldpername))
                print(f'Finish mean lcld raster {strftime("%m-%d-%Y %H:%M:%S")}')
            else:
                print(f'Mean lcld cover along flow accumulation already created {lcldPer_path}')
            stop = time.time()
            elapsed = int(stop - start)
            print(f'{filename[:4]} Mean lcld along FAC for {rname} complete - Elapsed time: ({datetime.timedelta(seconds=elapsed)})')
            print ("")
    # End Timing
    iter_time_stop = time.time()
    iter_time = int(iter_time_stop-iteration_start)
    print(f'{rname} fac weights complete - Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
    print('----------')
process_stop = time.time()
process_time = int(process_stop - process_start)
print(f'All regions complete - Elapsed time: ({datetime.timedelta(seconds=process_time)})')



Cook_Inlet
Begin Process (01-26-2022 12:03:57)

Finish glacier weighted flow accumulation at 01-26-2022 13:14:03
Finish percent glacier raster 01-26-2022 14:42:26
Percent Glacier cover along FAC for Cook_Inlet complete - Elapsed time: (2:38:28)

Begin Process (01-26-2022 14:42:26)

Finish lake weighted flow accumulation at 01-26-2022 15:56:29
Finish percent lake raster 01-26-2022 18:24:51
Percent lake cover along FAC for Cook_Inlet complete - Elapsed time: (3:42:24)

Begin Process (01-26-2022 18:24:51)

Finish slope weighted flow accumulation at 01-26-2022 21:09:52
Finish mean slope raster 01-27-2022 00:55:27
Mean slope cover along FAC for Cook_Inlet complete - Elapsed time: (6:30:36)

Begin Process (01-27-2022 00:55:27)

Finish north weighted flow accumulation at 01-27-2022 04:08:03
Finish percent north raster 01-27-2022 06:24:56
Percent north cover along FAC for Cook_Inlet complete - Elapsed time: (5:29:29)

Begin Process (01-27-2022 06:24:56)



### Extract to outlets of catchments outlets for catchments with a